In [0]:
%pip install -U -qqqq -r requirements.txt
%restart_python

In [0]:
%run ./function_calling_agent_openai_sdk

In [0]:
# Pydantic class to make configuration easiser to use.  Developers can use this, Python dictionaries or YAML files for their configuration.
from configs import (
    AgentConfig,
    FunctionCallingLLMConfig,
    LLMParametersConfig,
    RetrieverToolConfig,
    RetrieverParametersConfig,
    RetrieverSchemaConfig,
    UCToolConfig,
)
import yaml


docs_retriever = RetrieverToolConfig(
    vector_search_index="agents_demo.data.product_docs_index",  # UC Vector Search index
    vector_search_schema=RetrieverSchemaConfig(
        primary_key="product_id",
        chunk_text="indexed_doc",
        document_uri="product_id",
        additional_metadata_columns=[
            # "issue_area",
            # "issue_category",
            # "issue_sub_category",
            # "product_category",
            # "product_sub_category",
            # "conversation",
            # "timestamp",
            # "user_id",
        ],
    ),
    vector_search_parameters=RetrieverParametersConfig(
        num_results=1,
        query_type="ann",  # Type of search: ann or hybrid
    ),
    vector_search_threshold=0.0,
    # Tool prompt templates
    # chunk_template="Passage text: {chunk_text}\nPassage metadata: {metadata}\n\n",
    # prompt_template="""Use the following pieces of retrieved context to answer the question.\nOnly use the passages from context that are relevant to the query to answer the question, ignore the irrelevant passages.  When responding, cite your source, referring to the passage by the columns in the passage's metadata.\n\nContext: {context}""",
    retriever_query_parameter_prompt="query to look up in the product documentation",
    retriever_filter_parameter_prompt="Optional filters to apply to the search. An array of objects, each specifying a field name and the filters to apply to that field.  ONLY use the LIKE type of filter if you have a string to query in product_category, etc.  Prefer to query WITHOUT filters.",
    tool_description_prompt="Search the production documentation for product information.  If you need to know how to troubleshoot, what a product does, common issues, etc, use this tool.",
    tool_name="retrieve_product_docs",
    # Retriever internals
    tool_class_name="VectorSearchRetriever",
)

# python_exec_config = UCToolConfig(
#     uc_catalog_name="ep",
#     uc_schema_name="agent_demo",
#     uc_function_name="python_exec",
# )

# recent_orders = UCToolConfig(
#     uc_catalog_name="ep",
#     uc_schema_name="agent_demo",
#     uc_function_name="user_orders",
# )

########################
#### ✅✏️ LLM configuration
########################

llm_config = FunctionCallingLLMConfig(
    llm_endpoint_name="agents-demo-gpt4o",  # Model serving endpoint
    llm_system_prompt_template=(
        """You are a helpful assistant that answers questions by calling tools.  Provide responses ONLY based on the outputs from tools.  Ask follow up questions if needed.  If don't get relevant results from the retriever tool while using filters, try to call the retriever tool again with JUST a query and no filters!"""
    ),  # System prompt template
    llm_parameters=LLMParametersConfig(
        temperature=0.01, max_tokens=1500
    ),  # LLM parameters
    tools=[docs_retriever],
)

function_calling_agent_config = AgentConfig(
    llm_config=llm_config,
    input_example={
        "messages": [
            {
                "role": "user",
                "content": "What is the top customer issue?",
            },
        ]
    },
    agent_description="Has access to the product documentation, transcripts from our customer service call center and information about customer's recent orders.",
    agent_name="CustomerServiceTranscripts",
    endpoint_name="agents_ep-agent_demo-customer_bot_function_calling_agent",
)

with open("config.yml", "w") as file:
    yaml.dump(function_calling_agent_config.dict(), file, default_flow_style=False)

import json
print(json.dumps(function_calling_agent_config.dict(), indent=4))

In [0]:
agent = FunctionCallingAgent(agent_config=function_calling_agent_config.dict())

response = agent.predict(
    model_input={
        "messages": [
            {
                "role": "user",
                "content": "How does our blender work?",
            },
        ]
    }
)

In [0]:
from mlflow.models.resources import (
    DatabricksVectorSearchIndex,
    DatabricksServingEndpoint,
)
from mlflow.models.signature import ModelSignature
from mlflow.models.rag_signatures import StringResponse, ChatCompletionRequest, Message
import yaml
from databricks import agents
from databricks import vector_search
from databricks.vector_search.client import VectorSearchClient
from mlflow.types.llm import CHAT_MODEL_INPUT_SCHEMA

def log_agent_to_mlflow(agent_config, agent_code_file):
    # Add the Databricks resources so that credentials are automatically provisioned by agents.deploy(...)
    databricks_resources = [
        DatabricksServingEndpoint(
            endpoint_name=agent_config.llm_config.llm_endpoint_name
        ),
    ]

    # Add the Databricks resources for the retriever's vector indexes
    for tool in agent_config.llm_config.tools:
        if type(tool) == RetrieverToolConfig:
            databricks_resources.append(
                DatabricksVectorSearchIndex(index_name=tool.vector_search_index)
            )
            index_embedding_model = (
                VectorSearchClient(disable_notice=True)
                .get_index(index_name=tool.vector_search_index)
                .describe()
                .get("delta_sync_index_spec")
                .get("embedding_source_columns")[0]
                .get("embedding_model_endpoint_name")
            )
            if index_embedding_model is not None:
                databricks_resources.append(
                    DatabricksServingEndpoint(endpoint_name=index_embedding_model),
                )
            else:
                print(
                    "Could not identify the embedding model endpoint resource for {tool.vector_search_index}.  Please manually add the embedding model endpoint to `databricks_resources`."
                )

    # Specify the full path to the Agent notebook
    # model_file = "function_calling_agent_openai_sdk"
    # model_path = os.path.join(os.getcwd(), model_file)
    with open("requirements.txt", "r") as file:
        requirements = [line.strip() for line in file.readlines()]
    # Log the agent as an MLflow model
    return mlflow.pyfunc.log_model(
        python_model=agent_code_file,
        model_config=agent_config.dict(),
        artifact_path="agent",
        input_example=agent_config.input_example,
        resources=databricks_resources,
        signature=ModelSignature(
            inputs=CHAT_MODEL_INPUT_SCHEMA,
            outputs=StringResponse(),
        ),
        extra_pip_requirements=requirements+['pyspark'],
    )

In [0]:
with mlflow.start_run():
  logged_agent_info = log_agent_to_mlflow(function_calling_agent_config, "function_calling_agent_openai_sdk")

In [0]:
from databricks import agents
import mlflow

# You can log a new version or deploy an already logged/evaluated model from above.  Here, we use the last model logged for simplicity.

# Use Unity Catalog as the model registry
mlflow.set_registry_uri("databricks-uc")

uc_model_name = "agents_demo.playground.demo_agent"

# Register the Agent's model to the Unity Catalog
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, 
    name=uc_model_name # Unity Catalog model is configured in settings cell
)

# Deploy to enable the Review App and create an API endpoint
deployment_info = agents.deploy(uc_model_name, uc_registered_model_info.version)

displayHTML(
    f'<a href="{deployment_info.review_app_url}" target="_blank"><button style="color: white; background-color: #0073e6; padding: 10px 24px; cursor: pointer; border: none; border-radius: 4px;">SME Chat UI (review app)</button></a>'
)

displayHTML(
    f'<a href="{deployment_info.endpoint_url}" target="_blank"><button style="color: white; background-color: #0073e6; padding: 10px 24px; cursor: pointer; border: none; border-radius: 4px;">Model Serving REST API</button></a>'
)

In [0]:
%sql select * from agents_demo.default.product_docs_agent_payload_request_logs